---

# Code pour les flags 5 à 10



---

In [7]:
import subprocess

#on donne en argument à sign le message à signer et le chemin de la clé privée à utiliser pour signer.
def sign(plain_text,chemin_priv_key):
    # prépare les arguments à envoyer à openssl
    args = ['openssl', 'dgst', '-sha256', '-sign', chemin_priv_key,'-hex']

    # fichier=open('sign_elec.txt','r')
    message=plain_text
    # si plaintext est de stype str, on est obligé de l'encoder en bytes pour
    # pouvoir l'envoyer dans le pipeline vers openssl
    if isinstance(message, str):
        message=message.encode('utf-8')

    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=message, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    if error_message != '':
        print (error_message)
    

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    return result.stdout



# FLAG5 
dict.atk
(labo nord ouest, salle rc-13)

D'après le post-it on sait que l'utilisateur vienna5 a choisi son mdp (pour se connecter à son ordi fixe) dans la base de données cambridge_words.txt (dispo à la bibliothèque). 
De plus, d'après le feuillet on a la trace d'un protocole chap (voir docu de la bibliothèque) de l'utilisateur :  
- le challenge à déchiffrer
- le message clair correspondant au chiffré

Stratégie : faire tous les mots du Cambridge_words.txt et voir si le déchiffrage du chiffré (du feillet) à l'aide du mot de passe selectionné correspond au message clair (du feuillet).

In [1]:
import subprocess


#passphrase =ISECR0XX
#cryptext = U2FsdGVkX1/PqMju8jHsLbrm2hYBIukOzyGgBTK50ETxm6vde+w5uwR3gxZmMfBoW+7+ZxLbUrsZmiv/rFxFpg==
def decrypt2(passphrase,chiffre,cipher='aes-128-cbc'):
    # prépare les arguments à envoyer à openssl
    pass_arg = 'pass:{}'.format(passphrase)
    args = ['openssl', 'enc','-d','-base64','-' + cipher, '-pbkdf2', '-pass', pass_arg]
    # fichier = open("challenge_vienna5_chiffre.txt", "r")
    # chiffre=fichier.read()

    # si plaintext est de stype str, on est obligé de l'encoder en bytes pour
    # pouvoir l'envoyer dans le pipeline vers openssl
    if isinstance(chiffre, str):
        chiffre=chiffre.encode('utf-8')

    
    # ouvre le pipeline vers openssl. envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    result = subprocess.run(args, input=chiffre, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = result.stderr.decode()
    # if error_message != '':
    #     raise ValueError(error_message)
    
    # fichier.close()

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    # return result.stdout.strip()
    if error_message == '':
        return result.stdout.decode(errors='ignore')
    else :
        return 0

In [2]:


def brute_force_vienna5():
    fichier2=open('challenge_vienna5_clair.txt','r')
    message=fichier2.readlines()
    # print(message[0])
    fichier=open('cambridge_words.txt','r')
    lignes = fichier.readlines()  # Retourne une liste
    for ligne in lignes:
        # print(ligne[0:-1])
        if (decrypt2(ligne[0:-1],"U2FsdGVkX19ulWDW9FnSKWngn2VWswEtd9gXmQHLa2dyTle3+Z8VeW3qybpl3CVw\n",cipher='aes-128-cbc')==message[0]):
        # if (decrypt2(ligne[0:-1],message[0],cipher='aes-128-cbc')==message[1]):
            print("victoire")
            return(ligne[0:-1])
 


#Il faut laisser le \n dans le chiffré, cassandra l'a laissé
print(brute_force_vienna5())

victoire
availer


# FLAG 6 : 
chap.login

Authentification CHAP pour sécuriser la connexion au lancement du jeu (voir docu à la bibliothèque), attention ne pas oublier le \n à la fin du challenge !!

In [5]:

decrypt2("hellocrypto","U2FsdGVkX1+DPLptl1K4LHtWtXks1DEX57zBX/JWpaiq8Lo3OJl5FYi2Uj8F/ZCR\n",cipher='aes-128-cbc')

'raver scalp dhows guile totes'

# FLAG7 
pk.login

connexion super securisée grâce au protocole  _ PK _
Principe : on nous donne un message en clair à signer. Le jeu a notre clé publique en mémoire (cf flag3) et peut vérifier notre signature.

In [8]:

print("signature :", sign("decaf whelk moist gluts doser","keys/priv_key.pem"))

signature : b'SHA2-256(stdin)= a2128ba7b1a54ede8ed8f8ea7ac91d95f012fc79b181d2a4a8de689294c4e6db0787ba717125abdd9c15ee91e0e492357b4ee0cf3b5df7b34bfa0c0a209e4c5aad5fc2a54b7e2b8f8ea741bc0f3986951199c2a689e7117c5721a725659ab606e3ed8973310aaaacf4ed1c64c1d7c1f9d041a28f9cd8cc4b1954cfddb4592dd800240a3dc4a694d4d76bebd2f090e33fc0b88b51c44e771ccd90ee7f032b71b991242087099df26c4c805a1870aad1eb790a692c2c0752084d2f69e7d84cb00d5001f42538b73fc9c740f1644efc39edf53961d5aaa67537acb8f7fa3e0bc6d783e4b38eb7cc31902c73174e9171af991b18a381c606553ee7f2a69c9e9358af\n'


# Flag8 : 
pki.cert

Il suffit de copier le certificat de l'autorité de certification UGLIX et de l'upload dans ma card data dans un terminal de service dans le labo par exemple. Enfin, on se rend au lecteur nfc du cicsu et on tappe "ouvrir lecteur carte".

# Flag 9 (tme1)
tme.generator

In [9]:


import random
import sympy

def generate_prime(a, b, q):
    
    #Génère un nombre premier p tel que p = 1 + k*q avec k pair et a <= p < b.
    while True:
        #ici on gère le cas où a est pair ou impair sinon ça marche qu'une fois sur 2.
        if (a%2):
            k = random.randrange(((a-1)//q)+1, (b - 2) // q, 2)
        else : 
            k = random.randrange(((a-1)//q), (b - 2) // q, 2)
        
        p = 1 + k * q
        if a <= p < b and sympy.isprime(p):
            return p

def find_generator(p, q):
#Trouve un générateur g d'ordre q modulo p.
    while True:
        x = random.randint(2, p - 2)
        g = pow(x, (p - 1) // q, p)
        if g != 1:
            return g
        

a=int("7b32ef51c8ebaabf44f7f3c08fe163c4df90c5d0313c2c8ee280d558622d49c5718414fe817def4022cb59792f67153bdef9b0bb61e3e76e579c935fb8b7b08412be6494977940e71050712fb1d50b789b74919228f3e38d347fd4d529c63b9dd42e3d508d1c57f86bfeb6e0d2907be8a51a9ce15def406daa39e4fece59e97522a8f427f8ddb422e3e67d10f4d56534d55e6e1a3f03c402367b0b218b6e59b0598ca6cd33003b0d08b736e22f9ea8b9aff2f6dca296c53c40baae0bdff8902885010281f36da2aab6a4354360c717a1ef465b0597b49ec75fe20b96c825e073830b92458e5c8f06fd5771bde5bb1fda6ba3a94d5ac9afa9b966ace0962ef447",16)
b = a + 2**1950
q=int("9c1340b6df3e7e0e6fb1fe2a1fdaafbcf9f7738942b68d9539550e03c5f0599f",16)
p = generate_prime(a, b, q)
print("nombre premier trouvé")
g = find_generator(p, q)

print("p",p)
print("g",g)


nombre premier trouvé
p 15552428417381349189161127554324521796001441556843113555213565416350968479132987696392369975156876207979815336516677423305893681146432321998845799658592818607451225642579025510958801839788443720097981203987487235702216540547680257544535421137141658138028650822699004353419753822578791815164631208624840536835856182560833908880476657702402480160748461635019838167342872244631151563243454439294620209654438273834084835131966979580134651291723714563050409045189558593445630385587410243749478021934252641247027824103146646317892226115482705015669594801193243875906315196682861881835111135719686753323385331562101890225461
g 49923781232308086479240555202289872564847762756454623373803474221726143854200438488222757738753611981967384358201334005215657853053306900677091749471751718628713720779929598130837767040397932380628490637287539601426549231641318985367766382959680934707127952193425851414272433777963052111934925901428746727201775378782480533042297073891545398588103151308063

# TME2 FLAG 10
tme.primitive

In [10]:
import random

# Paramètres de départ
entier_a = int("f1e0c8816320352c5000c9186d80f11d4de8c6e3b48a7fab1d30c3fac7c6a8cf66d70507ba449fd42d0a4597ddccfaf76148dce53f29e1236d7dc753edff5fa2673422731ebc3628b824e8dc5657e526613ec9fa607e9d5077ff7e17c867ab8d843977ef3ad9268ddbf84b961124e959111cc2866f12d616129579958129c0697d87aace3406b653b8da5777282ea852a901e1fec1a27eb51a2e1b72eda9a18820a7e64dfbbc68c0c1eab610fc57eb1c8b668903fad1809d4a85aafead89558c059f410b415c72b0a382edd21ace578c8c26b6681b8404be9bffd3ce430b217adad26ef5d78a83b83a98c4b940455d12fff4a92cff3431d1915752d2649dcb30", 16)
entier_b = entier_a + 2**1950
q_ref = int("c3f344c6174f077d8b79974dfa3d7e7e3e5f33bf9758f52e53d8a51786b8bc2bea0a6acda098bd37c42a62c029ae519b", 16)

# print("entier_a =", entier_a)
# print("entier_b =", entier_b)
# print("q_ref =", q_ref)

# Test de primalité de Miller-Rabin
def miller_rabin(n, iterations):
    if (n == 2):
        return True
    if (n % 2 == 0 or n < 2):
        return False

    r = 0
    s = n - 1
    while (s % 2 == 0):
        r += 1
        s //= 2
    # print("r =", r)
    # print("s =", s)

    compteur_test = 0
    while (compteur_test < iterations):
        base = random.randrange(2, n - 1)
        x = pow(base, s, n)
        # print("base =", base)
        # print("x =", x)
        if (x != 1 and x != n - 1):
            repetition = 0
            while ((repetition < r - 1) and (x != n - 1)):
                x = pow(x, 2, n)
                repetition += 1
                # print("repetition =", repetition)
                # print("x (après carré) =", x)
            if (x != n - 1):
                return False
        compteur_test += 1

    return True

# Taille d'un entier en octets
def taille_octets(n):
    return (n.bit_length() + 7) // 8

# Conversion entier en tableau d'octets
def entier_vers_octets(n):
    return n.to_bytes(taille_octets(n), 'big')

# Algorithme d'Euclide étendu
def euclide_etendu(a, b):
    if (a == 0):
        return b, 0, 1
    pgcd, x1, y1 = euclide_etendu(b % a, a)
    return pgcd, y1 - (b // a) * x1, x1


constante_c = 10000
taille_bits_b = taille_octets(entier_b)
q_max_possible = (entier_b - entier_a) // (constante_c * (taille_bits_b ** 2))
taille_q = taille_octets(q_max_possible)

# print("taille_bits_b =", taille_bits_b)
# print("q_max_possible =", q_max_possible)
# print("taille_q =", taille_q)

# Construction de Q'
q_prime = 2 * q_ref
facteurs = [2]

while (taille_octets(q_prime) != taille_q):
    borne = min(q_ref - 1, 2 ** (8 * (taille_q - taille_octets(q_prime))))
    # print("borne =", borne)
    candidat = random.randrange(2, borne)
    # print("candidat =", candidat)
    if (miller_rabin(candidat, 10)):
        q_prime *= candidat
        facteurs.append(candidat)
        # print("q_prime =", q_prime)
        # print("facteurs =", facteurs)

a_reduit = entier_a // q_prime
b_reduit = entier_b // q_prime

# print("a_reduit =", a_reduit)
# print("b_reduit =", b_reduit)

# Recherche de pk
trouve_pk = False
while (trouve_pk == False):
    pk = random.randrange(a_reduit, b_reduit)
    # print("pk =", pk)
    if (miller_rabin(pk, 2)):
        p_candidat = q_prime * pk + 1
        # print("p_candidat =", p_candidat)
        if (miller_rabin(p_candidat, 2)):
            Q = q_prime * pk
            facteurs.append(pk)
            p = p_candidat
            trouve_pk = True
            # print("Q =", Q)
            # print("p =", p)

# Recherche d’un générateur g
essais_max = 200
index_essai = 0
generateur_trouve = False

while ((generateur_trouve == False) and (index_essai < essais_max)):
    g_candidat = random.randrange(2, p - 1)
    # print("g_candidat =", g_candidat)
    est_generateur = True
    index_verif = 0

    while ((index_verif < len(facteurs)) and (est_generateur)):
        facteur = facteurs[index_verif]
        # print("facteur =", facteur)
        if (pow(g_candidat, Q // facteur, p) == 1):
            est_generateur = False
            # print("échec avec facteur =", facteur)
        index_verif += 1

    if (est_generateur):
        g = g_candidat
        print("g =", g)
        generateur_trouve = True

    index_essai += 1
    # print("index_essai =", index_essai)

print("facteurs de (p-1)/q =", str(facteurs)[1:-1])


g = 2968390978971683130510499519982537452706492921113976427418079715355707112338953534005546616536068929064738394430243109698262927890138766298217533201493965203362017014799103552831767162309633705442035657621382469762973381505501982306188465255175410513987365861981314851264368519896209802414044502503612907402821218731538659034277054580824238622417755419159082242044018854944989743098679068204529685895821494821606693769119729637303045923858837798431347856017699997994909816126111445079204578067659274595223799375557484737558116309040507374743361995624509296097027034504863312059272128787269894335006912604644569509975
facteurs de (p-1)/q = 2, 23561213213014014782061864924857945961050199356802323389290801371559684172849412492090908722520280665163090933976809, 6372922862422069413235766210783383930302138919100052966988380765577535826880093984156083563737957816656848922549313, 3285847259338752642286138945047870447017449664168779758651284159005104141531482191410688683957595010266535233215959, 11